In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("harvest_cleaned_dataset.csv")
df.head()

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage,source
0,F00000001,188.0,1,0,1,0.0,0.0,0.0,1,0,train
1,F00000003,209.0,1,0,1,0.0,0.0,0.0,2,1,train
2,F00000004,257.0,1,0,1,0.0,0.0,0.0,2,1,train
3,F00000005,257.0,1,1,1,0.0,0.0,0.0,2,1,train
4,F00000006,342.0,1,0,1,0.0,0.0,0.0,2,1,train


In [5]:
df['Crop_Damage'].value_counts()

0    74238
1    12307
2     2313
Name: Crop_Damage, dtype: int64

In [9]:
df_damage2 = df[df['Crop_Damage'] == 2]

In [10]:
df_damage2

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage,source
7,F00000010,577.000000,1,0,1,0.0,0.0,0.0,1,2,train
9,F00000020,1132.000000,1,0,1,0.0,0.0,0.0,1,2,train
14,F00000029,1575.000000,1,1,1,0.0,0.0,0.0,2,2,train
103,F00000193,625.000000,0,1,3,30.0,29.0,0.0,1,2,train
112,F00000217,732.000000,0,0,3,50.0,30.0,0.0,2,2,train
...,...,...,...,...,...,...,...,...,...,...,...
88470,F00155221,3165.000000,0,1,2,40.0,40.0,18.0,1,2,train
88479,F00155242,3516.000000,0,1,2,25.0,50.0,4.0,3,2,train
88493,F00155265,1358.089433,0,1,2,60.0,48.0,10.0,1,2,train
88552,F00155357,448.000000,1,0,2,20.0,20.0,11.0,2,2,train


In [11]:
df_damage2.to_csv('harvest_damage_2.csv',index=False)

In [3]:
#Importing libraries
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,roc_auc_score,roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from scipy.stats import zscore
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB

In [4]:
df = df.drop(columns=['ID','source'])

In [5]:
df_xc=df.drop(columns=['Crop_Damage'])
yc=df[["Crop_Damage"]]
print(df_xc.head())
print(yc.head())

   Estimated_Insects_Count  Crop_Type  Soil_Type  Pesticide_Use_Category  \
0                    188.0          1          0                       1   
1                    209.0          1          0                       1   
2                    257.0          1          0                       1   
3                    257.0          1          1                       1   
4                    342.0          1          0                       1   

   Number_Doses_Week  Number_Weeks_Used  Number_Weeks_Quit  Season  
0                0.0                0.0                0.0       1  
1                0.0                0.0                0.0       2  
2                0.0                0.0                0.0       2  
3                0.0                0.0                0.0       2  
4                0.0                0.0                0.0       2  
   Crop_Damage
0            0
1            1
2            1
3            1
4            1


In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
xc = sc.fit_transform(df_xc)
df_xc=pd.DataFrame(xc,columns=df_xc.columns)

In [7]:
#defining a function to find accuracy score, crossvalidation score for the given dataset
def max_acc_score(names,model_c,df_xc,yc):
    accuracy_scr_max = 0
    for r_state in range(42,100):
        train_xc,test_xc,train_yc,test_yc = train_test_split(df_xc,yc,random_state = r_state,test_size = 0.33,stratify = yc)
        model_c.fit(train_xc,train_yc)
        accuracy_scr = accuracy_score(test_yc,model_c.predict(test_xc))
        if accuracy_scr> accuracy_scr_max:
            accuracy_scr_max=accuracy_scr
            final_state = r_state
            final_model = model_c
            mean_acc = cross_val_score(final_model,df_xc,yc,cv=5,scoring="accuracy").mean()
            std_dev = cross_val_score(final_model,df_xc,yc,cv=5,scoring="accuracy").std()
            cross_val = cross_val_score(final_model,df_xc,yc,cv=5,scoring="accuracy")
    print('\033[1m',"Results for model : ",names,'\n','\033[0m'
          "max accuracy score is" , accuracy_scr_max ,'\n',
          "Mean accuracy score is : ",mean_acc,'\n',
          "Std deviation score is : ",std_dev,'\n',
          "Cross validation scores are :  " ,cross_val) 
    print(" "*100)

In [8]:
#Now by using multiple Algorithms we are calculating the best Algo which suit best for our data set 
accuracy_scr_max = []
accuracy=[]
std_dev=[]
mean_acc=[]
cross_val=[]
models=[]
#models.append(('Random Forest', RandomForestClassifier()))
models.append(('KNN', KNeighborsClassifier()))
#models.append(('Decision Tree Classifier', DecisionTreeClassifier()))
#models.append(('Gaussian NB',GaussianNB()))


for names,model_c in models:
    max_acc_score(names,model_c,df_xc,yc)

 Results for model :  KNN 
 max accuracy score is 0.8312303914882008 
 Mean accuracy score is :  0.8278939494737061 
 Std deviation score is :  0.0013376548817429035 
 Cross validation scores are :   [0.82995724 0.82731263 0.82584965 0.82825952 0.82809071]
                                                                                                    


In [9]:
kNN=KNeighborsClassifier()
parameters={"n_neighbors":range(2,30)}
clf = GridSearchCV(kNN, parameters, cv=5,scoring="accuracy")
clf.fit(df_xc,yc)
clf.best_params_

{'n_neighbors': 20}

In [10]:
#Again running KNeighborsClassifier with n_neighbor = 20
kNN=KNeighborsClassifier(n_neighbors=20)
max_acc_score("KNeighbors Classifier",kNN,df_xc,yc)

 Results for model :  KNeighbors Classifier 
 max accuracy score is 0.8442913654344564 
 Mean accuracy score is :  0.8421301224991421 
 Std deviation score is :  0.0014742338024393855 
 Cross validation scores are :   [0.84396804 0.84284267 0.84115463 0.8428901  0.83979517]
                                                                                                    


In [11]:
xc_train,xc_test,yc_train,yc_test=train_test_split(df_xc, yc,random_state = 80,test_size=0.20,stratify=yc)
kNN.fit(xc_train,yc_train)
yc_pred=kNN.predict(xc_test)

In [27]:
yc_pred

array([0, 0, 0, ..., 0, 0, 0])

In [28]:
nnn = pd.DataFrame(yc_pred)

In [1]:
nnn.describe()

NameError: name 'nnn' is not defined

In [13]:
new = [[188,1,0,1,0,0,0,1]]

In [14]:
Ynew=kNN.predict(new)

In [15]:
Ynew

array([0])

In [16]:
import joblib
joblib.dump(kNN, "harvest_outcome_model.pkl")

['harvest_outcome_model.pkl']

In [17]:
data = [[188,1,0,1,0,0,0,1]]

In [18]:
model=joblib.load("harvest_outcome_model.pkl")
print(model.predict(data))

[0]


In [25]:
data = [[342,1,0,1,0,0,0,2]]

In [26]:
model=joblib.load("harvest_outcome_model.pkl")
print(model.predict(data))

[0]
